In [1]:
import numpy as np
import pandas as pd
import datetime
import mysql.connector

In [2]:
import mysql.connector
class user_info(object):
    
    # --------------------------------------- CONSTRUCTOR --------------------------------------
    def __init__(self, user_name, user_type= ""):
        
        # ---- CONSTANTS ----
        self.HOST = "localhost"
        self.DATABASE = "amit1"
        self.USER = "root"
        self.PASSWORD = "Amitsa1!"
        self.CONNECTION_CLOSED = "MySQL connection is closed"
        
        self.RIDER = "RIDER"
        self.REQUESTER = "REQUESTER"
        self.RIDER_TABLE = "rider"
        self.RIDER_USER_TABLE = "rider_user"
        self.REQUESTER_TABLE = "requester"
        self.REQUESTER_USER_TABLE = "requester_user"
        
        # Messages From APIs
        # Password Messages
        self.PASSWORD_MISMATCH = "PASSWORD MISMATCHED!"
        self.WRONG_PASSWORD = "WRONG PASSWORD!"
        
        # User Messages
        self.USER_CREATED = "USER CREATED!"
        self.USER_ALREADY_EXIST = "USER ALREADY EXIST!"
        self.USER_NOT_FOUND = "USER NOT FOUND!"
        self.WRONG_USER_NAME = "WRONG USER NAME!"
        
        # Login Messages
        self.LOGIN = "YOU LOGIN!" 
        self.ALREADY_LOGIN = "YOU ALREADY LOGIN!"
        self.PLEASE_LOGIN_MESSAGE = "PLEASE LOGIN, BEFORE CREATING REQUEST!"
        
        self.RECORD_EXIST = "RECORD ALREADY EXIST!"
        self.NO_MATCHING_RECORD = "NO MATCHING RECORD!"
        
        # ---- Store User Name ----
        self.user_name = user_name
    
        # ---- Store User Type ----
        if user_type.upper() == self.RIDER: 
            self.user_type = self.RIDER
        else: 
            self.user_type = self.REQUESTER
            
        # ---- is user login or not to the website ----
        self.user_login_or_not = False
        
        self.out = ""
        
        
    # ------------------------------- Create A Connection To The Database ------------------------------
    def make_connection(self):
        try:
            self.connection = mysql.connector.connect(host=self.HOST, database=self.DATABASE,user=self.USER , password=self.PASSWORD)
            if self.connection.is_connected():
                db_info = self.connection.get_server_info()
                self.cursor = self.connection.cursor(buffered=True)
                self.cursor.execute("select database();")               # Connected database
                record = self.cursor.fetchone()
                print(f"You're connected to database: {record[0]}")     # print Connected database name
        except Exception as e:
            print(f"Error while connecting to MySQL {e}")
            
            
    # ------------------------------- Disconnect The Connection ------------------------------
    def dis_connect(self):
        if self.connection.is_connected():
            self.cursor.close()
            self.connection.close()
        return self.CONNECTION_CLOSED
    
        
    # ------------------------------- Verify Password -------------------------------
    def password_varification(self, user_password1, user_password2):
        if user_password1 == user_password2:
            return True
        else: return self.PASSWORD_MISMATCH
        
    
    # ----------------------- Craete a user(Requester / Rider) --------------------------
    def create_user(self, first_name, last_name, user_password1, user_password2):
        valid_password = self.password_varification(user_password1, user_password2)
        if self.user_login_or_not:
            return self.USER_ALREADY_CREATED
        if valid_password != True:
            return valid_password
        
        self.first_name = first_name
        self.last_name = last_name
        
        self.make_connection()
        
        if self.connection.is_connected():
            try:
                self.cursor.execute(f"use {self.DATABASE};")
                if self.user_type == self.RIDER:
                    self.cursor.execute(f"SELECT user_name FROM {self.RIDER_USER_TABLE} WHERE user_name = '{self.user_name}'")
                else:
                    self.cursor.execute(f"SELECT user_name FROM {self.REQUESTER_USER_TABLE} WHERE user_name = '{self.user_name}'")
                record = self.cursor.fetchall()  
                if len(record) == 0:
                    if self.user_type == self.RIDER: sql = f"INSERT INTO {self.RIDER_USER_TABLE} (user_name, first_name, last_name, user_password) values (%s, %s, %s, %s)"
                    else: sql = f"INSERT INTO {self.REQUESTER_USER_TABLE} (user_name, first_name, last_name, user_password) values (%s, %s, %s, %s)"
                    val = (self.user_name, self.first_name, self.last_name, user_password1)
                    self.cursor.execute(sql, val)
                    self.connection.commit()
                    self.user_login_or_not = True
                    return self.USER_CREATED
                else:
                    return self.USER_ALREADY_EXIST
            except Exception as e:
                print("exception : ", e)
            finally:
                return self.dis_connect()

    # -------------------- If user is already present, Then login to the account ----------------------     
    def login_user(self, user_name, user_password):
        if self.user_name != user_name:
            return self.WRONG_USER_NAME
        if self.user_login_or_not == True:
            return self.ALREADY_LOGIN
        self.make_connection()
        if self.connection.is_connected():
            try:
                self.cursor.execute(f"use {self.DATABASE};")
                if self.user_type == self.RIDER:
                    self.cursor.execute(f"SELECT * FROM {self.RIDER_USER_TABLE} WHERE user_name = '{user_name}'")
                else:
                    self.cursor.execute(f"SELECT * FROM {self.REQUESTER_USER_TABLE} WHERE user_name = '{user_name}'")
                record = self.cursor.fetchone() 
                if record is None:
                    print(self.dis_connect())
                    return self.USER_NOT_FOUND
                if record[-1] == user_password: 
                    self.user_login_or_not = True
                    return self.LOGIN
                else: 
                    return self.WRONG_PASSWORD
                print("record ", record)
            except Exception as e:
                print("exception ", e)
            finally:
                print(self.dis_connect())
                
                
    # ------------------------------- Create Request API -------------------------------
    def create_request(self, src="", destination="", deliver_date="", flexible_timimg=False, no_of_assets=-1, 
                       asset_type="", asset_sensitivity="", whom_to_deliver=""):
        if self.user_type != self.REQUESTER:
            print("This Method is only available for REQUESTER")
            return False
        if self.user_login_or_not != True:
            return self.PLEASE_LOGIN_MESSAGE
        self.make_connection()
        if self.connection.is_connected():
            try:
                
                # Check the record is already present or not
                self.cursor.execute(f"use {self.DATABASE};")
                self.cursor.execute(f'''SELECT * FROM {self.REQUESTER_TABLE} WHERE user_name = '{self.user_name}' and src = '{src}' 
                and destination = '{destination}' and no_of_assets = "{no_of_assets}" and asset_type = "{asset_type}" 
                and asset_sensitivity = '{asset_sensitivity}' and whom_to_deliver = "{whom_to_deliver}" 
                and flexible_timimg = {flexible_timimg}''')
                record = self.cursor.fetchall() 
                if len(record) != 0:
                    filter_record = []
                    for sub_record in record:
                        if str(sub_record[4]) == deliver_date:
                            print("MATCHED_RECORD : ", record)
                            return self.RECORD_EXIST
                else:
                    print(self.NO_MATCHING_RECORD)
                        
                # Create Request
                sql = f'''INSERT INTO {self.REQUESTER_TABLE} (user_name, src, destination, deliver_date, flexible_timimg, no_of_assets, 
                asset_type, asset_sensitivity, whom_to_deliver, asset_status) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                val = (self.user_name, src, destination, deliver_date, flexible_timimg, no_of_assets, asset_type, asset_sensitivity, whom_to_deliver, "PENDING")
                self.cursor.execute(sql, val)
                self.connection.commit()
                print("Request Created!")
                
                # Return True
                return True
            except Exception as e:
                print("\nRequest not Created!")
                print(e.msg, "\n")
            finally:
                print(self.dis_connect())
                
            
    # ------------------------------- Share Travel Info Of Rider -------------------------------    
    def share_travel_info_of_rider(self, src="", destination="", deliver_date="", flexible_timimg=False, no_of_assets=0, travel_medium=""):
        if self.user_type != self.RIDER:
            print("This Method is only available for RIDER")
            return False
        if self.user_login_or_not != True:
            return self.PLEASE_LOGIN_MESSAGE
        self.make_connection()
        
        if self.connection.is_connected():
            try:
                self.cursor.execute(f"use {self.DATABASE};")
                self.cursor.execute(f'''SELECT * FROM {self.RIDER_TABLE} WHERE user_name = '{self.user_name}' and src = '{src}' 
                and destination = '{destination}' and travel_medium = "{travel_medium}" and asset_quantity = "{no_of_assets}" 
                and flexible_timimg = {flexible_timimg}''')
                record = self.cursor.fetchall() 
                if len(record) != 0:
                    filter_record = []
                    for sub_record in record:
                        if str(sub_record[4]) == deliver_date:
                            print("MATCHED_RECORD : ", record)
                            return "Record Already Exist!"
                else:
                    print("NO MATCHING RECORD!")
                    
                sql = f"INSERT INTO {self.RIDER_TABLE} (user_name, src, destination, deliver_date, flexible_timimg, travel_medium, asset_quantity, asset_status) values (%s, %s, %s, %s, %s, %s, %s, %s)"
                val = (self.user_name, src, destination, deliver_date, flexible_timimg, travel_medium, no_of_assets, "NOT_APPLIED")
                self.cursor.execute(sql, val)
                self.connection.commit()
                print("Shared Travel Information!")
                return True
            except Exception as e:
                print("\nRequest not Created!")
                print(e.msg, "\n")
            finally:
                print(self.dis_connect())
                
                
    # ------------------------------- User's Requests -------------------------------    
    def users_request(self):
        if self.user_type != self.REQUESTER:
            print("This Method is only available for REQUESTER")
            return False
        if self.user_login_or_not != True:
            print("Please Login, Before seeing Users Request!")
            return False
        self.make_connection()
        if self.connection.is_connected():
            try:
                
                # Fetch all Request of the user
                self.cursor.execute(f"use {self.DATABASE};")
                self.cursor.execute(f"SELECT * FROM {self.REQUESTER_TABLE} WHERE user_name = '{self.user_name}' ORDER BY deliver_date")
                record = self.cursor.fetchall() 
                if record is None:
                    print(self.dis_connect())
                    return "Record not Found"
                
                # update Status Column
                filter_record = []
                for sub_record in record:
#                     print("sub_rec", sub_record)
                    sub_record = list(sub_record)
                    if sub_record[-1] == "CONFIRM":
                        filter_record.append(sub_record)
                        continue
                    if str(datetime.datetime.now() - sub_record[4])[0] == "-":
                        if sub_record[-1] != "PENDING":
                            self.cursor.execute(f"UPDATE {self.REQUESTER_TABLE} SET asset_status = 'PENDING' WHERE request_id= {sub_record[0]}")
                            self.connection.commit()
                            sub_record[-1] = "PENDING"
                    else:
                        if sub_record[-1] != "EXPIRED":
                            self.cursor.execute(f"UPDATE {self.REQUESTER_TABLE} SET asset_status = 'EXPIRED' WHERE request_id= {sub_record[0]}")
                            self.connection.commit()
                            sub_record[-1] = "EXPIRED"
                    filter_record.append(sub_record)
                return pd.DataFrame(filter_record, columns=["request_id", "user_name", "src", "destination", "deliver_date", "flexible_timimg", "no_of_assets", "asset_type", "asset_sensitivity", "whom_to_deliver", "status"])
            except Exception as e:
                print("\nRequest not Created!")
                print(e.msg, "\n")
            finally:
                print(self.dis_connect()) 
                
    
    # ------------------------------- Shared Travel Information By Riders -------------------------------
    def shared_travel_info_by_rider(self):
        if self.user_type != self.RIDER:
            print("This Method is only available for RIDER")
            return False
        if self.user_login_or_not != True:
            print("Please Login, Before seeing Users Request!")
            return False
        self.make_connection()
        if self.connection.is_connected():
            try:
                
                # Fetch all Request of the user
                self.cursor.execute(f"use {self.DATABASE};")
                self.cursor.execute(f"SELECT * FROM {self.RIDER_TABLE} WHERE user_name = '{self.user_name}' ORDER BY deliver_date")
                record = self.cursor.fetchall() 
                if record is None:
                    print(self.dis_connect())
                    return "Record not Found"
                return pd.DataFrame(record, columns=["travel_id", "user_name", "src", "destination", "deliver_date", "flexible_timimg",  "travel_medium", "asset_quantity",  "status"])
            except Exception as e:
                print("\nRequest not Created!")
                print(e.msg, "\n")
            finally:
                print(self.dis_connect()) 
                
    
    # ---------------------------- Matched User's Request With Rider's Travel Information ---------------------------
    def matched_user_request(self):
        if self.user_type != self.REQUESTER:
            print("This Method is only available for REQUESTER")
            return False
        return self.matched_user_req()
    
    
    
    # ------------------------------- Apply From Matched Requests -------------------------------
    def apply_matched_request(self):
        if self.user_type != self.REQUESTER:
            print("This Method is only available for REQUESTER")
            return False
        return self.matched_user_req(only_match = False)
                
            
    # ------------------------------- Handle Previous 2 APIs -------------------------------
    def matched_user_req(self, only_match=True): 
        if self.user_login_or_not != True:
            print("Please Login, Before seeing Users Request!")
            return False
        self.make_connection()
        if self.connection.is_connected():
            try:
                self.cursor.execute(f"use {self.DATABASE};")
                
                # Fetch user's all requests
                all_request = f"SELECT * FROM {self.REQUESTER_TABLE} WHERE user_name = '{self.user_name}'"
                self.cursor.execute(all_request)
                record = self.cursor.fetchall() 
                
                # Match User's Request With Rider's Travel Information
                #    1. join on the basis of src, destination and datetime
                #    2. filter user_name, no_of_asset <= asset_quantity, 
                #    3. filter req.asset_status = "PENDING" and rid.asset_status = "NOT_APPLIED"
                #    4. Order By req.request_id
                all_request = f'''SELECT DISTINCT req.request_id, rid.travel_id FROM {self.REQUESTER_TABLE} as req 
                INNER JOIN {self.RIDER_TABLE} as rid ON req.src = rid.src and req.destination = rid.destination 
                and req.deliver_date = rid.deliver_date WHERE req.user_name="{self.user_name}" 
                and req.no_of_assets <= rid.asset_quantity and req.asset_status = "PENDING" 
                and rid.asset_status = "NOT_APPLIED" ORDER BY req.request_id;'''
                self.cursor.execute(all_request)
                record2 = self.cursor.fetchall() 
                if len(record2) == 0:
                    print("NO MATCHED REQUESTS!")
                    print(self.dis_connect())
                    return False
                dict_record = {}
                for sub_record in record2:
                    if dict_record.get(sub_record[0]) is None:
                        dict_record[sub_record[0]] = sub_record[1]
                dict_record[-1] = -1
                matched_keys = tuple(dict_record.keys())
                matched_values = tuple(dict_record.values())
                
                if only_match:
                    self.cursor.execute(f"SELECT * FROM {self.REQUESTER_TABLE} WHERE request_id in {matched_keys}")
                    record2 = self.cursor.fetchall() 
                    filter_record = []
                    for sub_record in record:
                        sub_record = list(sub_record)
                        if sub_record[0] in matched_keys:
                            sub_record[-1] = "MATCHED"
                        else:
                            sub_record[-1] = "NOT MATCHED"
                        filter_record.append(sub_record)
                    df = pd.DataFrame(filter_record, columns=["request_id", "user_name", "src", "destination", "deliver_date", "flexible_timimg", "no_of_assets", "asset_type", "asset_sensitivity", "whom_to_deliver", "status"])
                    self.out = df
                    return self.out
                else:
                    self.cursor.execute(f"UPDATE {self.REQUESTER_TABLE} SET asset_status = 'CONFIRM' WHERE request_id in {matched_keys}")
                    self.connection.commit()
                    self.cursor.execute(f"UPDATE {self.RIDER_TABLE} SET asset_status = 'APPLIED ' WHERE travel_id in {matched_values}")
                    self.connection.commit()
                    print("updated!")
                    df = self.users_request()
                    self.out = df
                    return self.out
                
            except Exception as e:
                print("\nRequest not Created!")
                print(e.msg, "\n")
            finally:
                print(self.dis_connect())
                return self.out
    def output(self):
        return self.out
                
            
user = user_info("amit_chourey")
print(user)
print(user.create_user("Amit", "Chourey", "Amit!@#$", "Amit!@#$"))
print(user.login_user("amit_chourey", "Amit!@#$"))

print("\n")

user2 = user_info("rider5", "rider")
print(user2)
print(user2.create_user("Rider", "5", "Rider!@#$", "Rider!@#$"))
print(user2.login_user("rider5", "Rider!@#$"))

You're connected to database: amit1
MySQL connection is closed
You're connected to database: amit1
MySQL connection is closed
YOU LOGIN!


You're connected to database: amit1
MySQL connection is closed
You're connected to database: amit1
MySQL connection is closed
YOU LOGIN!


In [3]:
user.create_request(src="NarmadaPuram", destination="Manali", deliver_date="2022-12-20 16:00:56", flexible_timimg=False, no_of_assets=4, asset_type="LAPTOP", asset_sensitivity="HIGHLY_SENSITIVE", whom_to_deliver="pradeep_meena")
# print("\n")
# user.users_request()

You're connected to database: amit1
NO MATCHING RECORD!
Request Created!
MySQL connection is closed


True

In [4]:
user2.share_travel_info_of_rider(src="NarmadaPuram", destination="Manali", deliver_date="2022-12-20 16:00:56", flexible_timimg=True, no_of_assets=4, travel_medium="CAR")
# print("\n")
# user.users_request()

You're connected to database: amit1
NO MATCHING RECORD!
Shared Travel Information!
MySQL connection is closed


True

In [5]:
user.users_request()

You're connected to database: amit1
MySQL connection is closed


,request_id,user_name,src,destination,deliver_date,flexible_timimg,no_of_assets,asset_type,asset_sensitivity,whom_to_deliver,status
0,10,amit_chourey,NarmadaPuram,Pandurna,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
1,19,amit_chourey,NarmadaPuram,Mathura,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
2,20,amit_chourey,NarmadaPuram,vrindavan,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
3,21,amit_chourey,NarmadaPuram,Hyderabad,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
4,22,amit_chourey,NarmadaPuram,Chandigarh,2022-12-11 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,PENDING
5,23,amit_chourey,NarmadaPuram,Vienna,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,PENDING
6,25,amit_chourey,NarmadaPuram,Manali,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,PENDING


In [6]:
user2.shared_travel_info_by_rider()

You're connected to database: amit1
MySQL connection is closed


,travel_id,user_name,src,destination,deliver_date,flexible_timimg,travel_medium,asset_quantity,status
0,7,rider5,NarmadaPuram,Pandurna,2022-12-04 16:00:56,1,CAR,4,NOT_APPLIED
1,8,rider5,NarmadaPuram,Vienna,2022-12-20 16:00:56,1,CAR,4,NOT_APPLIED
2,10,rider5,NarmadaPuram,Manali,2022-12-20 16:00:56,1,CAR,4,NOT_APPLIED


In [7]:
user.matched_user_request()
user.output().drop(["request_id"], axis=1)

You're connected to database: amit1
MySQL connection is closed


,user_name,src,destination,deliver_date,flexible_timimg,no_of_assets,asset_type,asset_sensitivity,whom_to_deliver,status
0,amit_chourey,NarmadaPuram,Pandurna,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,NOT MATCHED
1,amit_chourey,NarmadaPuram,Mathura,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,NOT MATCHED
2,amit_chourey,NarmadaPuram,vrindavan,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,NOT MATCHED
3,amit_chourey,NarmadaPuram,Hyderabad,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,NOT MATCHED
4,amit_chourey,NarmadaPuram,Chandigarh,2022-12-11 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,MATCHED
5,amit_chourey,NarmadaPuram,Vienna,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,MATCHED
6,amit_chourey,NarmadaPuram,Manali,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,MATCHED


In [8]:
user.apply_matched_request()
user.output().drop(["request_id"], axis=1)

You're connected to database: amit1
updated!
You're connected to database: amit1
MySQL connection is closed
MySQL connection is closed


,user_name,src,destination,deliver_date,flexible_timimg,no_of_assets,asset_type,asset_sensitivity,whom_to_deliver,status
0,amit_chourey,NarmadaPuram,Pandurna,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
1,amit_chourey,NarmadaPuram,Mathura,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
2,amit_chourey,NarmadaPuram,vrindavan,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
3,amit_chourey,NarmadaPuram,Hyderabad,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
4,amit_chourey,NarmadaPuram,Chandigarh,2022-12-11 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,CONFIRM
5,amit_chourey,NarmadaPuram,Vienna,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,CONFIRM
6,amit_chourey,NarmadaPuram,Manali,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,CONFIRM


In [86]:
user.users_request().drop(["request_id"], axis=1)

You're connected to database: amit1
MySQL connection is closed


,user_name,src,destination,deliver_date,flexible_timimg,no_of_assets,asset_type,asset_sensitivity,whom_to_deliver,status
0,aditya_chourey,Jhanshi,Pune,2022-12-15 12:45:56,1,3,TRAVEL_BAG,SENSITIVE,akshat_singh,CONFIRM
1,aditya_chourey,NarmadaPuram,Vienna,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,PENDING


You're connected to database: amit1
MySQL connection is closed


,request_id,user_name,src,destination,deliver_date,flexible_timimg,no_of_assets,asset_type,asset_sensitivity,whom_to_deliver,status
0,10,amit_chourey,NarmadaPuram,Pandurna,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
1,19,amit_chourey,NarmadaPuram,Mathura,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
2,20,amit_chourey,NarmadaPuram,vrindavan,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
3,21,amit_chourey,NarmadaPuram,Hyderabad,2022-12-04 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,chaitanya_lohe,EXPIRED
4,22,amit_chourey,NarmadaPuram,Chandigarh,2022-12-11 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,CONFIRM
5,23,amit_chourey,NarmadaPuram,Vienna,2022-12-20 16:00:56,0,4,LAPTOP,HIGHLY_SENSITIVE,pradeep_meena,CONFIRM
